In [2]:
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("/home/anon/projects2/GNNcoal/")
#from graphseq_inference.data_utils import *
from GNNcoal.models import *
#from graphseq_inference.train_utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
import os
import tskit
import numpy as np
import pandas as pd

In [6]:
directory = "./csvs/20k_dataset/"

files = os.listdir(directory)
files = [directory +  file for file in files]


len(files)

FileNotFoundError: [Errno 2] No such file or directory: './csvs/20k_dataset/'

In [11]:
def convert_tree_sequence_to_data_object_alpha_r(tree_sequence: tskit.trees.TreeSequence,
                                                     parameters: np.ndarray,
                                                     num_trees:int = 500,
                                                     num_embedding:int = 60, 
                           ):
    
    alpha = parameter_set.model
    y = torch.Tensor([alpha])
    
    tree_sequence = reduce_tree_sequence(tree_sequence, 3)

    
    max_num_nodes = 2 * tree_sequence.num_samples - 1 
    data_objects = []
    
    for i, tree in enumerate(tree_sequence.trees()):
        if i < num_trees:
            
            data = from_networkx(nx.Graph(tree.as_dict_of_dicts()))
            rename_data_attribute(data, "branch_length", "edge_weight") 
            num_nodes = data.num_nodes
            data.x = torch.eye(max_num_nodes,num_embedding)
            data.x[num_nodes:] = torch.zeros(num_embedding)
            data.y = torch.Tensor(y)
            data.num_nodes = max_num_nodes
            data_objects.append(data)
            
        else: 
            break

        
    return data_objects

In [18]:
pwd

'/home/anon/projects2/GNNcoal-analysis/training_notebooks'

In [19]:
parameters = pd.read_csv("../csvs/20k_seed_0x1337_demographies.csv")

In [21]:
def get_alpha(file): return float(file.split("_")[-1].replace(".trees", ""))

In [22]:
from os.path import basename


class AlphaInferenceModel(nn.Module):
    
    def __init__(self, DemographyNet, time_window=60):
        super().__init__()
        self.l1 = nn.Linear(time_window, time_window//2)
        self.l2 = nn.Linear(time_window//2, time_window//4)
        self.l3 = nn.Linear(time_window//4, 1)
        self.DemographyNet = DemographyNet
    
    def forward(self, batch):
        x = self.DemographyNet(batch)
        return self.l3(F.relu(self.l2(F.relu(self.l1(x)))))



In [25]:
def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))


In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
demography_net = DiffPoolNet(19, 60, 192, 60, track_running_stats=False)
model = AlphaInferenceModel(demography_net)
model = model.to(device)
criterion = RMSELoss
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [27]:
length = 1
loss_all = []

for epoch in range(0, 10):
    np.random.shuffle(files)
    for i, file in enumerate(tqdm(files)):


        ts, mask = torch.load(file)
        #ts = tskit.load(file)
        #ts = tskit.load(file)
        #data_objects = convert_tree_sequence_to_data_object_alpha(ts, get_alpha(file))
        nth_scenario = int(file.split("_")[2])
        parameter_set = parameters.iloc[nth_scenario]
        data_objects = convert_tree_sequence_to_data_object_alpha_r(ts, parameter_set)

        if len(data_objects) > 1:
        
            #mask[population_time <= 10] = False
            #mask = torch.tile(torch.Tensor(mask), (len(data_objects), 1))

            optimizer.zero_grad()
            dl = DataLoader(data_objects, batch_size=len(data_objects))
            for batch in dl:
                batch = batch.to(device)
                y_hat = model(batch)
                y_true = data_objects[0].y.tile(len(data_objects)).reshape(len(data_objects), length).to(device)

                loss = criterion(y_hat, y_true) 
                loss.backward()
                loss_all.append(loss.item())
                optimizer.step()

            if i != 0 and i % 10000 == 0:
                loss_all = np.mean(loss_all)
                print(f"loss {loss_all}")
                torch.save(model.state_dict(), "./alpha_inf_M3/mmc_diffpool_model_alpha_inference_intermediate" + str(epoch) + "_" + str(i) + ".pth")
                os.system(f'echo "Epoch: {epoch:03d}, Train Loss: {np.mean(loss_all):.4f}" >> ./alpha_inf_M3/mmc_diffpool_model_alpha_inference_intermediate.txt')
                loss_all = []
                
torch.save(model.state_dict(), "./alpha_inf_M3/mmc_diffpool_model_alpha_inference_intermediate" + str(epoch) + "_" + str(i) + ".pth")

NameError: name 'files' is not defined